In [ ]:
import cv2
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
sns.set_style('whitegrid')
from sklearn.metrics import confusion_matrix , classification_report
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Flatten , Conv2D , MaxPooling2D , Dropout , Activation , BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam , Adamax
from tensorflow.keras import regularizers

#Warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import tensorflow as tf
#tf.keras.mixed_precision.set_global_policy('mixed_float16')

import tensorflow as tf
import numpy as np
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization, ReLU, Add
from tensorflow.keras.models import Model
from tensorflow.keras.losses import KLDivergence
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications import DenseNet121, ResNet50V2
from tensorflow.keras.layers import GlobalAveragePooling2D
import copy
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization, ReLU, Add
from tensorflow.keras.models import Model
from tensorflow.keras.losses import KLDivergence
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications import DenseNet169, MobileNetV2, ResNet50, EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D
import copy

import tensorflow as tf
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras import layers


In [ ]:
X_train_s = np.load('X.npy')
y_train_s = np.load('Y.npy')

X_train_s.shape, y_train_s.shape

In [ ]:
from sklearn.model_selection import train_test_split

X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(X_train_s, y_train_s, test_size=0.2, random_state=42)
X_train_s, X_val_s, y_train_s, y_val_s = train_test_split(X_train_s, y_train_s, test_size=0.2, random_state=42)

X_train_s.shape,X_test_s.shape, y_train_s.shape,y_test_s.shape, y_val_s.shape,y_val_s.shape

In [ ]:
class GlobalAttentionNoiseLayer(layers.Layer):
    def __init__(self, noise_factor=0.2, **kwargs):
        super(GlobalAttentionNoiseLayer, self).__init__(**kwargs)
        self.noise_factor = noise_factor

    def build(self, input_shape):
        self.alpha = self.add_weight(
            shape=(1,),
            initializer='ones',
            trainable=True,
            name='alpha'
        )
        super(GlobalAttentionNoiseLayer, self).build(input_shape)

    def call(self, inputs, training=None):
        if training:
            noise = tf.random.normal(
                shape=tf.shape(inputs),
                mean=0.0,
                stddev=self.noise_factor * self.alpha
            )
            return inputs + noise
        else:
            return inputs


class FeatureAttentionNoiseLayer(layers.Layer):
    def __init__(self, noise_factor=0.2, learning_rate=0.001, **kwargs):
        super(FeatureAttentionNoiseLayer, self).__init__(**kwargs)
        self.noise_factor = noise_factor
        self.learning_rate = learning_rate

    def build(self, input_shape):
        _, H, W, C = input_shape
        self.alpha = self.add_weight(
            shape=(H, W, C),
            initializer='ones',
            trainable=True,
            name='alpha'
        )
        self.dense = layers.Dense(units=C, activation='relu')
        super(FeatureAttentionNoiseLayer, self).build(input_shape)

    def call(self, inputs, training=None):
        if training:
            noise = tf.random.normal(shape=tf.shape(inputs),
                                     mean=0.0,
                                     stddev=self.noise_factor)

            global_avg_pooled = tf.reduce_mean(inputs, axis=[1, 2], keepdims=True)
            delta_related = self.dense(global_avg_pooled)

            attention_noise = inputs + delta_related * inputs * self.alpha * noise
            # Optional regularization
            
            #self.add_loss(tf.reduce_mean(attention_noise))
            
            return attention_noise
        else:
            return inputs

    def get_config(self):
        config = super(FeatureAttentionNoiseLayer, self).get_config()
        config.update({
            'noise_factor': self.noise_factor,
            'learning_rate': self.learning_rate
        })
        return config



################################################ NORAN ################################################

class CombinedNoiseLayer(layers.Layer):
    def __init__(self, global_noise_factor=0.2, feature_noise_factor=0.2,
                 learning_rate=0.001, **kwargs):
        super(CombinedNoiseLayer, self).__init__(**kwargs)
        self.global_noise_layer = GlobalAttentionNoiseLayer(
            noise_factor=global_noise_factor
        )
        self.feature_noise_layer = FeatureAttentionNoiseLayer(
            noise_factor=feature_noise_factor,
            learning_rate=learning_rate
        )

    def call(self, inputs, training=None):
        if training:
            x = self.global_noise_layer(inputs, training=training)
            x = self.feature_noise_layer(x, training=training)
            return x
        else:
            return inputs

    def get_config(self):
        config = super(CombinedNoiseLayer, self).get_config()
        config.update({
            'global_noise_factor': self.global_noise_layer.noise_factor,
            'feature_noise_factor': self.feature_noise_layer.noise_factor,
            'learning_rate': self.feature_noise_layer.learning_rate
        })
        return config


# --- Spatial & Channel attention from your previous code --- #
from tensorflow.keras.layers import Layer, Conv2D, GlobalAveragePooling2D, Dense

class TrainableSpatialAttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(TrainableSpatialAttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.convolution = Conv2D(
            filters=1,
            kernel_size=(1, 1),
            activation='sigmoid',
            padding='same',
            trainable=False
        )
        super(TrainableSpatialAttentionLayer, self).build(input_shape)

    def call(self, inputs):
        attention_weights = self.convolution(inputs)
        return tf.multiply(inputs, attention_weights)


class TrainableChannelAttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(TrainableChannelAttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.global_avg_pooling = GlobalAveragePooling2D()
        self.dense1 = Dense(units=input_shape[-1] // 8,
                            activation='relu',
                            trainable=False)
        self.dense2 = Dense(units=input_shape[-1],
                            activation='sigmoid',
                            trainable=False)
        super(TrainableChannelAttentionLayer, self).build(input_shape)

    def call(self, inputs):
        avg_pool = self.global_avg_pooling(inputs)
        dense1_out = self.dense1(avg_pool)
        channel_attention_weights = self.dense2(dense1_out)
        channel_attention_weights = tf.expand_dims(
            tf.expand_dims(channel_attention_weights, 1), 1
        )
        return tf.multiply(inputs, channel_attention_weights)


###########################  RecRAN ##############################

class TrainableCombinedAttentionLayer(Layer):
    def __init__(self, spatial_noise_factor=0.6,
                 channel_noise_factor=0.6, num_layers=3, **kwargs):
        super(TrainableCombinedAttentionLayer, self).__init__(**kwargs)
        self.spatial_attentions = [
            TrainableSpatialAttentionLayer() for _ in range(num_layers)
        ]
        self.channel_attentions = [
            TrainableChannelAttentionLayer() for _ in range(num_layers)
        ]

        # Trainable weights for noise factors
        self.spatial_noise_weight = self.add_weight(
            shape=(1,),
            initializer='ones',
            trainable=False,
            name='spatial_noise_weight'
        )
        self.channel_noise_weight = self.add_weight(
            shape=(1,),
            initializer='ones',
            trainable=False,
            name='channel_noise_weight'
        )

        self.spatial_noise_factor = spatial_noise_factor
        self.channel_noise_factor = channel_noise_factor

    def call(self, inputs):
        spatial_attention_output = inputs
        channel_attention_output = inputs

        for spatial_attention, channel_attention in zip(
            self.spatial_attentions, self.channel_attentions
        ):
            spatial_attention_output = spatial_attention(spatial_attention_output)
            channel_attention_output = channel_attention(channel_attention_output)

        # Spatial & channel noise
        spatial_noise = tf.random.normal(
            shape=tf.shape(spatial_attention_output),
            mean=0,
            stddev=self.spatial_noise_factor
        )
        channel_noise = tf.random.normal(
            shape=tf.shape(channel_attention_output),
            mean=0,
            stddev=self.channel_noise_factor
        )

        spatial_noise *= self.spatial_noise_weight + (
            spatial_noise * self.spatial_noise_weight
        )
        channel_noise *= self.channel_noise_weight + (
            channel_noise * self.channel_noise_weight
        )

        spatial_attention_output += spatial_noise
        channel_attention_output += channel_noise

        spatial_attention_output = tf.clip_by_value(
            spatial_attention_output, 0.0, 1.0
        )
        channel_attention_output = tf.clip_by_value(
            channel_attention_output, 0.0, 1.0
        )

        combined_attention1 = tf.multiply(
            spatial_attention_output, channel_attention_output
        )
        combined_attention2 = tf.multiply(
            spatial_attention_output, channel_attention_output
        )
        combined_attention3 = tf.multiply(
            spatial_attention_output, channel_attention_output
        )
        combined_attention4 = tf.multiply(combined_attention1, combined_attention2)
        combined_attention = tf.multiply(combined_attention3, combined_attention4)

        return tf.multiply(inputs, combined_attention)


# ----------------------------------------------------------------------
# 3. SANI
# ----------------------------------------------------------------------

class SoftplusMergedAttentionNoiseLayer(layers.Layer):
    """
    Fuse two attention/noise mechanisms with softplus weights:

      - path 1: CombinedNoiseLayer (global + feature noise)
      - path 2: TrainableCombinedAttentionLayer (spatial + channel attention + noise)

    x_gf  = CombinedNoiseLayer(x, training=...)
    x_att = TrainableCombinedAttentionLayer(x)

    Δ_gf  = x_gf  - x
    Δ_att = x_att - x

    w_g = softplus(theta_g) >= 0
    w_a = softplus(theta_a) >= 0

    Δ = (w_g * Δ_gf + w_a * Δ_att) / (w_g + w_a + eps)
    y = x + Δ
    """

    def __init__(self,
                 global_noise_factor=0.2,
                 feature_noise_factor=0.2,
                 learning_rate=0.001,
                 spatial_noise_factor=0.6,
                 channel_noise_factor=0.6,
                 num_layers=3,
                 use_eval_noise=True,
                 eps=1e-6,
                 **kwargs):
        super(SoftplusMergedAttentionNoiseLayer, self).__init__(**kwargs)

        self.combined_noise = CombinedNoiseLayer(
            global_noise_factor=global_noise_factor,
            feature_noise_factor=feature_noise_factor,
            learning_rate=learning_rate
        )

        self.attention_noise = TrainableCombinedAttentionLayer(
            spatial_noise_factor=spatial_noise_factor,
            channel_noise_factor=channel_noise_factor,
            num_layers=num_layers
        )

        self.use_eval_noise = bool(use_eval_noise)
        self.eps = float(eps)

    def build(self, input_shape):
        self.theta_global_feat = self.add_weight(
            name=self.name + "_theta_global_feat",
            shape=(),
            initializer=tf.keras.initializers.Zeros(),
            trainable=True
        )
        self.theta_att = self.add_weight(
            name=self.name + "_theta_att",
            shape=(),
            initializer=tf.keras.initializers.Zeros(),
            trainable=True
        )
        super().build(input_shape)

    def call(self, inputs, training=None):
        #if training is None:
         #   training = tf.keras.backend.learning_phase()

        x = inputs

        # Path 1: global + feature noise
        if training:
            x_gf = self.combined_noise(x, training=True)
        else:
            if self.use_eval_noise:
                x_gf = self.combined_noise(x, training=True)   # force noise at inference
            else:
                x_gf = self.combined_noise(x, training=False)

        # Path 2: spatial + channel attention noise
        x_att = self.attention_noise(x)  # no 'training' flag in your code

        # Noise increments
        delta_gf = x_gf - x
        delta_att = x_att - x

        w_g = tf.nn.softplus(self.theta_global_feat)  # >=0
        w_a = tf.nn.softplus(self.theta_att)          # >=0

        denom = w_g + w_a + self.eps
        delta = (w_g * delta_gf + w_a * delta_att) / denom

        y = x + delta
        return y



In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.layers import Layer, Conv2D, GlobalAveragePooling2D, Dense

# ----------------------------------------------------------------------
# 0. Setup
# ----------------------------------------------------------------------
tf.keras.backend.clear_session()
tf.random.set_seed(0)
np.random.seed(0)

# ----------------------------------------------------------------------
# 1. Your attention code, but with trainable=False everywhere
# ----------------------------------------------------------------------


###########  Spatial\Local Attention ############
class TrainableSpatialAttentionLayer(Layer):
    """
    Your spatial attention, but convolution is NON-TRAINABLE.
    """
    def __init__(self, **kwargs):
        super(TrainableSpatialAttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.convolution = Conv2D(
            filters=1,
            kernel_size=(1, 1),
            activation='sigmoid',
            padding='same',
            trainable=False,                    # CHANGED
            name=self.name + "_conv"
        )
        super(TrainableSpatialAttentionLayer, self).build(input_shape)

    def call(self, inputs):
        attention_weights = self.convolution(inputs)
        return tf.multiply(inputs, attention_weights)


############# Channel Attention ###############

class TrainableChannelAttentionLayer(Layer):
    """
    Your channel attention, but Dense layers are NON-TRAINABLE.
    """
    def __init__(self, **kwargs):
        super(TrainableChannelAttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.global_avg_pooling = GlobalAveragePooling2D(name=self.name + "_gap")
        self.dense1 = Dense(
            units=input_shape[-1] // 8,
            activation='relu',
            trainable=False,                   # CHANGED
            name=self.name + "_dense1"
        )
        self.dense2 = Dense(
            units=input_shape[-1],
            activation='sigmoid',
            trainable=False,                   # CHANGED
            name=self.name + "_dense2"
        )
        super(TrainableChannelAttentionLayer, self).build(input_shape)

    def call(self, inputs):
        avg_pool = self.global_avg_pooling(inputs)
        dense1_out = self.dense1(avg_pool)
        channel_attention_weights = self.dense2(dense1_out)
        channel_attention_weights = tf.expand_dims(
            tf.expand_dims(channel_attention_weights, 1), 1
        )
        return tf.multiply(inputs, channel_attention_weights)


###########################  RecRAN ##############################

class TrainableCombinedAttentionLayer(Layer):
    """
    Exactly your TrainableCombinedAttentionLayer logic, except:
      - All internal layers are frozen (trainable=False).
      - spatial_noise_weight and channel_noise_weight are NON-TRAINABLE.
    """
    def __init__(self, spatial_noise_factor=0.6,
                 channel_noise_factor=0.6,
                 num_layers=3, **kwargs):
        super(TrainableCombinedAttentionLayer, self).__init__(**kwargs)
        self.spatial_attentions = [
            TrainableSpatialAttentionLayer(name=f"{self.name}_spat{i}")
            for i in range(num_layers)
        ]
        self.channel_attentions = [
            TrainableChannelAttentionLayer(name=f"{self.name}_chan{i}")
            for i in range(num_layers)
        ]

        # Noise factor weights – now NON-TRAINABLE
        self.spatial_noise_weight = self.add_weight(
            shape=(1,),
            initializer='ones',
            trainable=False,                  # CHANGED
            name=self.name + "_spatial_noise_weight"
        )
        self.channel_noise_weight = self.add_weight(
            shape=(1,),
            initializer='ones',
            trainable=False,                  # CHANGED
            name=self.name + "_channel_noise_weight"
        )

        self.spatial_noise_factor = float(spatial_noise_factor)
        self.channel_noise_factor = float(channel_noise_factor)

    def call(self, inputs):
        spatial_attention_output = inputs
        channel_attention_output = inputs

        for spatial_attention, channel_attention in zip(
            self.spatial_attentions, self.channel_attentions
        ):
            # Spatial Attention
            spatial_attention_output = spatial_attention(spatial_attention_output)

            # Channel Attention
            channel_attention_output = channel_attention(channel_attention_output)

        # Generate spatial and channel noise
        spatial_noise = tf.random.normal(
            shape=tf.shape(spatial_attention_output),
            mean=0.0,
            stddev=self.spatial_noise_factor
        )
        channel_noise = tf.random.normal(
            shape=tf.shape(channel_attention_output),
            mean=0.0,
            stddev=self.channel_noise_factor
        )

        # Scale the noise with (frozen) weights
        spatial_noise *= self.spatial_noise_weight + (
            spatial_noise * self.spatial_noise_weight
        )
        channel_noise *= self.channel_noise_weight + (
            channel_noise * self.channel_noise_weight
        )

        # Add spatial attention noise
        spatial_attention_output += spatial_noise

        # Add channel attention noise
        channel_attention_output += channel_noise

        # Clip attention maps to ensure they are within the valid range [0, 1]
        spatial_attention_output = tf.clip_by_value(spatial_attention_output, 0.0, 1.0)
        channel_attention_output = tf.clip_by_value(channel_attention_output, 0.0, 1.0)

        ########################### Combine attention mechanisms with recursuve nature #########################
        
        combined_attention1 = tf.multiply(spatial_attention_output, channel_attention_output)
        combined_attention2 = tf.multiply(spatial_attention_output, channel_attention_output)
        combined_attention3 = tf.multiply(spatial_attention_output, channel_attention_output)
        combined_attention4 = tf.multiply(combined_attention1, combined_attention2)
        combined_attention = tf.multiply(combined_attention3, combined_attention4)

        return tf.multiply(inputs, combined_attention)


# ----------------------------------------------------------------------
# 2. Conv layer with weight noise gated by your (frozen) attention-noise
# ----------------------------------------------------------------------

"""
Conv2D whose weights are randomized using your Frozen CombinedAttentionLayer.

For each forward pass:

    1. att_out = CombinedAttention(x)      # your code, frozen
    2. gate_in = mean(att_out, over N,H,W) # per-input-channel gate
    3. normalize gate_in to [0,1]
    4. eps ~ N(0, noise_std^2) same shape as W
    5. noise_gated = eps * gate_in_reshaped
    6. W_eff = W_base + rho * noise_gated
    7. y = Conv2D(x, W_eff) + BN + optional ReLU

- Combined attention is NON-TRAINABLE (frozen defense structure).
- rho (noise strength) is TRAINABLE and learned via backprop.
"""

'''class AttentionWeightNoiseConv2D(layers.Layer):
    

    def __init__(self,
                 filters,
                 kernel_size,
                 strides=1,
                 padding="same",
                 use_bias=False,
                 activation=True,
                 noise_std=0.05,
                 spatial_noise_factor=0.6,
                 channel_noise_factor=0.6,
                 att_num_layers=3,
                 use_eval_noise=True,
                 **kwargs):
        super().__init__(**kwargs)
        if isinstance(kernel_size, int):
            kernel_size = (kernel_size, kernel_size)

        self.filters = int(filters)
        self.kernel_size = kernel_size
        self.strides = strides
        self.padding = padding
        self.use_bias = use_bias
        self.activation = activation
        self.noise_std = float(noise_std)
        self.use_eval_noise = bool(use_eval_noise)

        # Base conv + BN + optional ReLU
        self.conv = Conv2D(
            filters=self.filters,
            kernel_size=self.kernel_size,
            strides=self.strides,
            padding=self.padding,
            use_bias=self.use_bias,
            kernel_initializer="he_normal",
            name=self.name + "_conv"
        )
        self.bn = layers.BatchNormalization(name=self.name + "_bn")
        self.act = layers.ReLU(name=self.name + "_relu") if self.activation else None

        # Your combined attention–noise, but frozen
        self.att = TrainableCombinedAttentionLayer(
            spatial_noise_factor=spatial_noise_factor,
            channel_noise_factor=channel_noise_factor,
            num_layers=att_num_layers,
            name=self.name + "_att"
        )

    def build(self, input_shape):
        # Build conv & BN
        self.conv.build(input_shape)
        conv_out_shape = self.conv.compute_output_shape(input_shape)
        self.bn.build(conv_out_shape)

        # Trainable noise strength (per conv layer)
        self.rho_logit = self.add_weight(
            name=self.name + "_rho_logit",
            shape=(),
            initializer=tf.keras.initializers.Constant(-2.0),  # small initial rho
            trainable=True
        )

        super().build(input_shape)

    def call(self, x, training=None):
        W_base = self.conv.kernel          # [kh,kw,C_in,C_out]
        strides_tf = [1, self.conv.strides[0], self.conv.strides[1], 1]
        padding_tf = self.conv.padding.upper()  # "SAME" or "VALID"

        # 1) Run your frozen combined attention on the features
        att_out = self.att(x)              # [N,H,W,C_in]

        # 2) Derive a per-input-channel gating vector from att_out
        gate_in = tf.reduce_mean(att_out, axis=[0, 1, 2])   # [C_in]
        # Normalize to [0,1] to keep noise scale stable
        gate_in = gate_in / (tf.reduce_max(tf.abs(gate_in)) + 1e-6)
        gate_in = tf.reshape(gate_in, [1, 1, -1, 1])        # [1,1,C_in,1]

        # 3) Sample Gaussian noise in weight space
        eps = tf.random.normal(
            shape=tf.shape(W_base),
            mean=0.0,
            stddev=self.noise_std
        )  # [kh,kw,C_in,C_out]

        # Gate noise per input channel using your attention-derived gate
        noise_gated = eps * gate_in

        # 4) Trainable noise ratio
        rho = tf.nn.softplus(self.rho_logit)  # >= 0

        # 5) Decide if noise is active
        #noise_on = bool(training) or self.use_eval_noise

        #if noise_on:
        
        W_eff = W_base + rho * noise_gated
        
        #else:
         #   #W_eff = W_base
          #  W_eff = W_base + rho * noise_gated

        # 6) Perform convolution with W_eff
        y = tf.nn.conv2d(x, W_eff, strides=strides_tf, padding=padding_tf)
        if self.conv.use_bias and self.conv.bias is not None:
            y = tf.nn.bias_add(y, self.conv.bias)

        # 7) BN + activation
        y = self.bn(y, training=training)
        if self.act is not None:
            y = self.act(y)
        return y'''



class AttentionWeightNoiseConv2D(layers.Layer):
    """
    Conv2D whose weights are randomized using BOTH:

      (1) Your TrainableCombinedAttentionLayer (spatial+channel attention+noise)
      (2) Your CombinedNoiseLayer (global + feature attention noise)

    Inside the conv, we build a Softplus-style fusion in *feature space*:

        x_gf  = CombinedNoiseLayer(x, training=...)
        x_att = TrainableCombinedAttentionLayer(x)

        Δ_gf  = x_gf  - x
        Δ_att = x_att - x

        w_g = softplus(theta_g) >= 0
        w_a = softplus(theta_a) >= 0

        x_fused = x + (w_g * Δ_gf + w_a * Δ_att) / (w_g + w_a + eps)

    Then we compress x_fused into a per-input-channel gate and use it to
    modulate Gaussian weight noise:

        gate_in = mean(x_fused, over N,H,W)   -> [C_in]
        noise   = N(0, noise_std^2) in weight space
        noise_gated = noise * gate_in (broadcasted to [kh,kw,C_in,C_out])

        W_eff = W_base + rho * noise_gated

    - SINGLE conv branch (no multi-branch, no multi-scale).
    - Uses your attention noise *and* global/feature noise.
    - Softplus mixing is trainable (theta_g, theta_a).
    """

    def __init__(self,
                 filters,
                 kernel_size,
                 strides=1,
                 padding="same",
                 use_bias=False,
                 activation=True,
                 noise_std=0.05,
                 spatial_noise_factor=0.6,
                 channel_noise_factor=0.6,
                 att_num_layers=3,
                 use_eval_noise=True,
                 **kwargs):
        super().__init__(**kwargs)
        if isinstance(kernel_size, int):
            kernel_size = (kernel_size, kernel_size)

        self.filters = int(filters)
        self.kernel_size = kernel_size
        self.strides = strides
        self.padding = padding
        self.use_bias = use_bias
        self.activation = activation
        self.noise_std = float(noise_std)
        self.use_eval_noise = bool(use_eval_noise)

        # Base conv + BN + optional ReLU
        self.conv = Conv2D(
            filters=self.filters,
            kernel_size=self.kernel_size,
            strides=self.strides,
            padding=self.padding,
            use_bias=self.use_bias,
            kernel_initializer="he_normal",
            name=self.name + "_conv"
        )
        self.bn = layers.BatchNormalization(name=self.name + "_bn")
        self.act = layers.ReLU(name=self.name + "_relu") if self.activation else None

        # Your combined attention–noise (spatial + channel), typically frozen
        self.att = TrainableCombinedAttentionLayer(
            spatial_noise_factor=spatial_noise_factor,
            channel_noise_factor=channel_noise_factor,
            num_layers=att_num_layers,
            name=self.name + "_att"
        )

        # NEW: global + feature attention noise (your CombinedNoiseLayer)
        # used ONLY to help shape the weight noise.
        self.combined_noise = CombinedNoiseLayer(
            global_noise_factor=0.2,
            feature_noise_factor=0.2,
            learning_rate=0.001
        )

    def build(self, input_shape):
        # Build underlying conv & BN
        self.conv.build(input_shape)
        conv_out_shape = self.conv.compute_output_shape(input_shape)
        self.bn.build(conv_out_shape)

        # Trainable noise strength (per conv layer)
        self.rho_logit = self.add_weight(
            name=self.name + "_rho_logit",
            shape=(),
            initializer=tf.keras.initializers.Constant(-2.0),  # small initial rho
            trainable=True
        )

        # NEW: Softplus mixing weights for the two noise paths
        self.theta_global_feat = self.add_weight(
            name=self.name + "_theta_global_feat",
            shape=(),
            initializer=tf.keras.initializers.Zeros(),
            trainable=True
        )
        self.theta_att_feat = self.add_weight(
            name=self.name + "_theta_att_feat",
            shape=(),
            initializer=tf.keras.initializers.Zeros(),
            trainable=True
        )

        super().build(input_shape)

    def call(self, x, training=None):
        W_base = self.conv.kernel          # [kh, kw, C_in, C_out]
        strides_tf = [1,
                      self.conv.strides[0],
                      self.conv.strides[1],
                      1]
        padding_tf = self.conv.padding.upper()  # "SAME" or "VALID"

        # ------------------------------------------------------------------
        # 1) Feature-space fusion of your two attention/noise mechanisms
        # ------------------------------------------------------------------
        # Path 1: global + feature noise
        if training or self.use_eval_noise:
            x_gf = self.combined_noise(x, training=True)
        else:
            x_gf = self.combined_noise(x, training=False)

        # Path 2: spatial + channel attention noise
        x_att = self.att(x)  # your TrainableCombinedAttentionLayer

        # Increments
        delta_gf = x_gf - x
        delta_att = x_att - x

        # Softplus mixing weights (>= 0)
        w_g = tf.nn.softplus(self.theta_global_feat)
        w_a = tf.nn.softplus(self.theta_att_feat)

        denom = w_g + w_a + 1e-6
        #x_fused = x + (w_g * delta_gf + w_a * delta_att) / denom
        x_fused = (w_g * delta_gf + w_a * delta_att) / denom

        # ------------------------------------------------------------------
        # 2) Turn fused features into a per-input-channel gate
        # ------------------------------------------------------------------
        # gate_in: [C_in]
        gate_in = tf.reduce_mean(x_fused, axis=[0, 1, 2])
        gate_in = gate_in / (tf.reduce_max(tf.abs(gate_in)) + 1e-6)
        gate_in = tf.reshape(gate_in, [1, 1, -1, 1])   # [1,1,C_in,1]

        # ------------------------------------------------------------------
        # 3) Sample Gaussian noise in weight space and gate it
        # ------------------------------------------------------------------
        eps = tf.random.normal(
            shape=tf.shape(W_base),
            mean=0.0,
            stddev=self.noise_std
        )  # [kh, kw, C_in, C_out]

        # Per-input-channel gating of weight noise
        noise_gated = eps * gate_in  # broadcasts to [kh,kw,C_in,C_out]

        # Trainable scale of weight noise
        rho = tf.nn.softplus(self.rho_logit)  # >= 0

        # Effective weights
        W_eff = W_base + rho * noise_gated

        # ------------------------------------------------------------------
        # 4) Convolution with W_eff + BN + optional activation
        # ------------------------------------------------------------------
        y = tf.nn.conv2d(x, W_eff, strides=strides_tf, padding=padding_tf)
        if self.conv.use_bias and self.conv.bias is not None:
            y = tf.nn.bias_add(y, self.conv.bias)

        y = self.bn(y, training=training)
        if self.act is not None:
            y = self.act(y)
        return y



# ----------------------------------------------------------------------
# 3. ResNet-18 style block using AttentionWeightNoiseConv2D
# ----------------------------------------------------------------------

############################# HySCAN which replaces standard convolution block   ##################################

class ResidualBlockWithWeightNoise(layers.Layer):
    """
    Basic ResNet-18 block:

        x -> Conv(noisy weights, stride)
           -> Conv(noisy weights, stride=1)
           -> + shortcut
           -> ReLU
    """
    def __init__(self, filters, stride=1, noise_std=0.05,
                 spatial_noise_factor=0.6, channel_noise_factor=0.6,
                 att_num_layers=3, **kwargs):
        super().__init__(**kwargs)
        self.filters = int(filters)
        self.stride = int(stride)
        self.noise_std = float(noise_std)
        self.spatial_noise_factor = float(spatial_noise_factor)
        self.channel_noise_factor = float(channel_noise_factor)
        self.att_num_layers = int(att_num_layers)

        ################################# RWAN as implicit stochasticity which replaces standard convolution layer   ###############################################
        
        self.conv1 = AttentionWeightNoiseConv2D(
            filters=self.filters,
            kernel_size=3,
            strides=self.stride,
            padding="same",
            use_bias=False,
            activation=True,
            noise_std=self.noise_std,
            spatial_noise_factor=self.spatial_noise_factor,
            channel_noise_factor=self.channel_noise_factor,
            att_num_layers=self.att_num_layers,
            use_eval_noise=True,
            name=self.name + "_conv1"
        )

        ################################# RWAN as implicit stochasticity which replaces standard convolution layer   ###############################################
        
        self.conv2 = AttentionWeightNoiseConv2D(
            filters=self.filters,
            kernel_size=3,
            strides=1,
            padding="same",
            use_bias=False,
            activation=False,
            noise_std=self.noise_std,
            spatial_noise_factor=self.spatial_noise_factor,
            channel_noise_factor=self.channel_noise_factor,
            att_num_layers=self.att_num_layers,
            use_eval_noise=True,
            name=self.name + "_conv2"
        )

        self.proj = None
        self.proj_bn = None

        ###################################### SANI as explicit stochasticity at the end of HySCAN block  ##########################################################
        
        self.attention = SoftplusMergedAttentionNoiseLayer(
        global_noise_factor=0.2,
        feature_noise_factor=0.2,
        spatial_noise_factor=0.6,
        channel_noise_factor=0.6,
        num_layers=3,
        use_eval_noise=True,
        #name="stage1_softplus_noise"
    )

    def build(self, input_shape):
        in_ch = int(input_shape[-1])
        if in_ch != self.filters or self.stride != 1:
            self.proj = Conv2D(
                filters=self.filters,
                kernel_size=1,
                strides=self.stride,
                padding="same",
                use_bias=False,
                kernel_initializer="he_normal",
                name=self.name + "_proj_conv"
            )
            self.proj_bn = layers.BatchNormalization(name=self.name + "_proj_bn")
        super().build(input_shape)

    def call(self, x, training=None):

        ################################# RWAN as implicit stochasticity which replaces standard convolution layer   ###############################################
        y = self.conv1(x, training=training)
        #y = self.attention(y)

        ################################# RWAN as implicit stochasticity which replaces standard convolution layer   ###############################################
        y = self.conv2(y, training=training)
        

        if self.proj is not None:
            shortcut = self.proj_bn(self.proj(x), training=training)
        else:
            shortcut = x

        out = tf.nn.relu(y + shortcut)

        ###################################### SANI as explicit stochasticity at the end of HySCAN block  ##########################################################
        out = self.attention(out)
        return out





def _make_stage(x, filters, num_blocks, stride, noise_std,
                spatial_noise_factor, channel_noise_factor,
                att_num_layers, name_prefix):


    ############################# HySCAN which replaces standard convolution block   ##################################
    
    x = ResidualBlockWithWeightNoise(
        filters=filters,
        stride=stride,
        noise_std=noise_std,
        spatial_noise_factor=spatial_noise_factor,
        channel_noise_factor=channel_noise_factor,
        att_num_layers=att_num_layers,
        name=name_prefix + "_block0"
    )(x)

    '''x = SoftplusMergedAttentionNoiseLayer(
        global_noise_factor=0.2,
        feature_noise_factor=0.2,
        spatial_noise_factor=0.3,
        channel_noise_factor=0.3,
        num_layers=3,
        use_eval_noise=True,
        #name="stage1_softplus_noise"
    )(x)'''

    ############################# HySCAN which replaces standard convolution block   ##################################
    
    for i in range(1, num_blocks):
        x = ResidualBlockWithWeightNoise(
            filters=filters,
            stride=1,
            noise_std=noise_std,
            spatial_noise_factor=spatial_noise_factor,
            channel_noise_factor=channel_noise_factor,
            att_num_layers=att_num_layers,
            name=f"{name_prefix}_block{i}"
        )(x)

    ###################################### SANI as explicit stochasticity at the end of HySCAN block  ##########################################################
    
    x = SoftplusMergedAttentionNoiseLayer(
    global_noise_factor=0.2,
    feature_noise_factor=0.2,
    spatial_noise_factor=0.6,
    channel_noise_factor=0.6,
    num_layers=3,
    use_eval_noise=True,
    #name="stage1_softplus_noise"
)(x)
    return x


# ----------------------------------------------------------------------
# 4. Full ResNet-18 backbone with attention-noisy weights
# ----------------------------------------------------------------------

def build_resnet18_attention_noisy_weights(input_shape=(128, 128, 3),
                                           num_classes=5,
                                           base_width=64,
                                           noise_std=0.05,
                                           spatial_noise_factor=0.6,
                                           channel_noise_factor=0.6,
                                           att_num_layers=3):
    """
    ResNet-18 topology [2,2,2,2] where every conv in every block is:
        AttentionWeightNoiseConv2D
    which uses your frozen TrainableCombinedAttentionLayer to shape
    weight noise.

    No spectral norm, no GroupSort.
    """
    inputs = tf.keras.Input(shape=input_shape)

    # Stem
    x = Conv2D(
        filters=base_width,
        kernel_size=3,
        strides=1,
        padding="same",
        use_bias=False,
        kernel_initializer="he_normal",
        name="stem_conv"
    )(inputs)
    x = layers.BatchNormalization(name="stem_bn")(x)
    x = layers.ReLU(name="stem_relu")(x)

    # ResNet-18 stages
    x = _make_stage(x, base_width,     num_blocks=2, stride=1,
                    noise_std=noise_std,
                    spatial_noise_factor=spatial_noise_factor,
                    channel_noise_factor=channel_noise_factor,
                    att_num_layers=att_num_layers,
                    name_prefix="stage1")

    x = _make_stage(x, base_width * 2, num_blocks=2, stride=2,
                    noise_std=noise_std,
                    spatial_noise_factor=spatial_noise_factor,
                    channel_noise_factor=channel_noise_factor,
                    att_num_layers=att_num_layers,
                    name_prefix="stage2")

    x = _make_stage(x, base_width * 4, num_blocks=2, stride=2,
                    noise_std=noise_std,
                    spatial_noise_factor=spatial_noise_factor,
                    channel_noise_factor=channel_noise_factor,
                    att_num_layers=att_num_layers,
                    name_prefix="stage3")

    x = _make_stage(x, base_width * 8, num_blocks=2, stride=2,
                    noise_std=noise_std,
                    spatial_noise_factor=spatial_noise_factor,
                    channel_noise_factor=channel_noise_factor,
                    att_num_layers=att_num_layers,
                    name_prefix="stage4")

    # Head
    x = layers.GlobalAveragePooling2D(name="avgpool")(x)
    outputs = layers.Dense(
        num_classes,
        activation="softmax",   # if you want logits, use activation=None and from_logits=True
        name="head_dense"
    )(x)

    model = tf.keras.Model(inputs, outputs, name="ResNet18_AttentionNoisyWeights")
    return model


# ----------------------------------------------------------------------
# 5. Example: build & compile
# ----------------------------------------------------------------------



In [ ]:
import tensorflow as tf

# ------------------------------------------------------------------
# 1️⃣  Hyper-parameters
# ------------------------------------------------------------------
sigma_train  = 0.5          # same σ you will use later for certification
batch_size   = 8
epochs       = 100


from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay

initial_gamma = 0.5
learning_rate = 1e-2
optimizer = Adam(learning_rate=0.001)
#opt = Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.9, epsilon=None, amsgrad=False)
# Compile the model with the custom optimizer
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              #loss_weights=[initial_gamma, (1 -  initial_gamma)],
              metrics=['accuracy'], jit_compile=True)


from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
def checkpoint_callback():

    checkpoint_filepath = 'best1_model_cer_skin_lung.keras'

    model_checkpoint_callback= ModelCheckpoint(filepath=checkpoint_filepath,
                           save_weights_only=False,
                           #frequency='epoch',
                           monitor='val_loss',
                           save_best_only=True,
                            mode='min',
                           verbose=1)

    return model_checkpoint_callback

def early_stopping(patience):
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', restore_best_weights=True, patience=60, verbose=1)
    return es_callback



from tensorflow.keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=50, verbose = 1, min_lr=0.00001)

checkpoint_callback = checkpoint_callback()

early_stopping = early_stopping(patience=100)
callbacks = [checkpoint_callback, early_stopping, reduce_lr]
            


# ------------------------------------------------------------------
# 2️⃣  Build tf.data pipelines
# ------------------------------------------------------------------
gauss = tf.keras.layers.GaussianNoise(stddev=sigma_train)

def add_noise(x, y):
    """
    Add i.i.d. Gaussian noise **only during training**.
    Keras will pass 'training=True' when this dataset
    is used for .fit; no noise when you later call model(x, training=False).
    """
    x = gauss(x, training=True)        # <- noise goes here
    return x, y

gauss = tf.keras.layers.GaussianNoise(stddev=sigma_train)

def add_noise(x, y):
    x = tf.cast(x, tf.float32) / 255.0       # ①  uint8 → float32  [0,1]
    x = gauss(x, training=True)              # ②  additive N(0,σ²)
    return x, y

def preprocess_val(x, y):
    x = tf.cast(x, tf.float32) / 255.0       # same cast, but NO noise
    return x, y


train_ds = (
    tf.data.Dataset.from_tensor_slices((X_train_s, y_train_s))
      .shuffle(len(X_train_s))
      .batch(batch_size)
      .map(add_noise,  num_parallel_calls=tf.data.AUTOTUNE)
      .prefetch(tf.data.AUTOTUNE)
)

val_ds = (
    tf.data.Dataset.from_tensor_slices((X_val_s, y_val_s))
      .batch(batch_size)
      .prefetch(tf.data.AUTOTUNE)      # **no** noise on validation set
)

# ------------------------------------------------------------------
# 3️⃣  Fit exactly as before
# ------------------------------------------------------------------
history = model.fit(
    train_ds,
    epochs         = epochs,
    validation_data= val_ds,
    verbose        = 1,
    callbacks      = callbacks,        # your ModelCheckpoint / EarlyStopping
    shuffle        = False             # shuffled already in the pipeline
)


In [ ]:
# hybrid_certifier.py
import tensorflow as tf, numpy as np
from scipy.stats import beta, norm

# ---------- One-sided lower Clopper–Pearson bound ----------
def lower_confidence_bound(k, n, alpha):
    # P(Bin(n, p) >= k) >= alpha  ⇒ lower bound on p
    return beta.ppf(alpha, k, n - k + 1) if k > 0 else 0.0

# ---------- Randomized smoothing ----------
class SmoothedClassifier:
    def __init__(self, base_model, sigma):
        """
        base_model must be DETERMINISTIC when called with training=False.
        sigma: std-dev of the Gaussian used BOTH for training & certification.
        """
        self.f = base_model
        self.sigma = float(sigma)
        self.num_classes = int(base_model.output_shape[-1])

    @tf.function(jit_compile=True)
    def _predict_noisy(self, x_batch):
        noise = tf.random.normal(tf.shape(x_batch)) * self.sigma
        logits = self.f(x_batch + noise, training=False)
        return tf.argmax(logits, axis=-1)

    def sample_class(self, x, N0=1000, batch=128):
        counts = np.zeros(self.num_classes, dtype=np.int32)
        reps = tf.repeat(x[None, ...], batch, axis=0)
        for done in range(0, N0, batch):
            m = min(batch, N0 - done)
            preds = self._predict_noisy(reps[:m])
            counts += np.bincount(preds.numpy(), minlength=self.num_classes)
        return int(np.argmax(counts))

    def certify(self, x, N=100_000, N0=1000, alpha=1e-3, batch=512):
        A = self.sample_class(x, N0=N0, batch=batch)
        countA = 0
        reps = tf.repeat(x[None, ...], batch, axis=0)
        for done in range(0, N, batch):
            m = min(batch, N - done)
            preds = self._predict_noisy(reps[:m])
            countA += int(np.sum(preds.numpy() == A))
        pA_low = lower_confidence_bound(countA, N, alpha)
        if pA_low <= 0.5:
            return 0.0
        return self.sigma * norm.ppf(pA_low)

# ---------- Deterministic (Lipschitz) branch ----------
def deterministic_radius(model, x, L=1.0):
    """
    r_det = margin / (2L), where margin = top1 - top2 logit.
    Only valid if your network has a PROVABLE Lipschitz constant ≤ L.
    """
    logits = model(x[None, ...], training=False)  # deterministic forward
    top2 = tf.nn.top_k(logits, k=2).values
    margin = float((top2[:, 0] - top2[:, 1])[0].numpy())
    return max(margin / (2.0 * float(L)), 0.0)

# ---------- Hybrid wrapper ----------
class HybridCertifier:
    def __init__(self, base_model, sigma, L=1.0):
        self.smooth = SmoothedClassifier(base_model, sigma)
        self.model  = base_model
        self.L      = float(L)

    @staticmethod
    def assert_deterministic(model, x, atol=1e-6):
        """Catches accidental randomness when training=False."""
        y1 = model(x[None, ...], training=False)
        y2 = model(x[None, ...], training=False)
        tf.debugging.assert_near(y1, y2, atol=atol,
            message="Model appears stochastic at eval; gate all noise with `if training:`.")

    def certify(self, x, N=100_000, N0=1000, alpha=1e-3, batch=512):
        # Optional health check (run once per script, not per sample)
        # self.assert_deterministic(self.model, x)

        r_det    = deterministic_radius(self.model, x, L=self.L)
        r_smooth = self.smooth.certify(x, N=N, N0=N0, alpha=alpha, batch=batch)
        return max(r_det, r_smooth), r_det, r_smooth


In [ ]:
# Build a clean test dataset like your val pipeline
def preprocess(x, y):
    #return tf.cast(x, tf.float32) / 255.0, y
    return x, y #tf.cast(x, tf.float32) / 255.0, y

test_ds = (tf.data.Dataset.from_tensor_slices((X_test_s, y_test_s))
             .batch(1)
             .map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
             .prefetch(tf.data.AUTOTUNE))

# HYBRID certification
from hybrid_certifier import HybridCertifier
SIGMA = 0.5     # MUST equal the sigma you used in GaussianNoise during training
L     = 1.0     # Set to 1.0 only if every layer is provably ≤1-Lipschitz

hybrid = HybridCertifier(model, sigma=SIGMA, L=L)

clean_ok = det_ok = smooth_ok = hybrid_ok = 0
N0, N, alpha = 1_000, 100_000, 1e-3

for x, y in test_ds:
    true = int(tf.argmax(y, -1).numpy()[0])
    pred = int(tf.argmax(model(x, training=False), -1).numpy()[0])
    clean_ok += int(pred == true)

    r_best, r_det, r_smooth = hybrid.certify(tf.squeeze(x, 0), N=N, N0=N0, alpha=alpha, batch=256)
    det_ok    += int((r_det    > 0) and (pred == true))
    smooth_ok += int((r_smooth > 0) and (pred == true))
    hybrid_ok += int((r_best   > 0) and (pred == true))

n_test = len(X_test_s)
print(f"\nClean acc              : {clean_ok/n_test:.3%}")
print(f"Deterministic cert acc : {det_ok  /n_test:.3%}")
print(f"Smoothing    cert acc  : {smooth_ok/n_test:.3%}")
print(f"HYBRID       cert acc  : {hybrid_ok/n_test:.3%}  (α={alpha}, σ={SIGMA})")


In [ ]:
# hybrid_certify.py
import tensorflow as tf, numpy as np, math
from pathlib import Path
from scipy.stats import beta, norm    # pip install scipy

# ----------------------------------------------------------------------
# 0.  Hyper-parameters
# ----------------------------------------------------------------------
SIGMA      = 0.5          # std-dev for input Gaussian noise  (train + certify)
LIPSCHITZ  = 1.0          # global Lipschitz constant of your network
EPOCHS     = 50
BATCH_TRAIN= 128
N0, N      = 1_000, 100_000   # Monte-Carlo sample counts (CIFAR-10 defaults)
ALPHA      = 1e-3             # failure prob → 99.9 % confidence
CKPT_FILE  = "hybrid_noise.keras"

# ----------------------------------------------------------------------
# 1.  Data  (CIFAR-10 for demo; swap in your own dataset)
# ----------------------------------------------------------------------

#(x_tr, y_tr), (x_te, y_te) = tf.keras.datasets.cifar10.load_data()
#C = 10
#x_tr = x_tr.astype("float32")/255.0; x_te = x_te.astype("float32")/255.0
#y_tr = tf.keras.utils.to_categorical(y_tr, C); y_te = tf.keras.utils.to_categorical(y_te, C)
#train_ds = tf.data.Dataset.from_tensor_slices((x_tr, y_tr)).shuffle(50_000).batch(BATCH_TRAIN)
#test_ds  = tf.data.Dataset.from_tensor_slices((x_te, y_te)).batch(1)  # certify one-by-one

# ----------------------------------------------------------------------
# 2.  Model  (⇢ put **your own architecture** here)
# ----------------------------------------------------------------------
#model      = make_model()
loss_fn    = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
optimizer  = tf.keras.optimizers.Adam(1e-3)
noise_layer= tf.keras.layers.GaussianNoise(SIGMA)      # input noise

# ----------------------------------------------------------------------
# 3.  Training loop (Gaussian-noise augmentation every batch)
# ----------------------------------------------------------------------
@tf.function(jit_compile=True)
def train_step(x, y):
    with tf.GradientTape() as tape:
        x_noisy = noise_layer(x, training=True)
        logits  = model(x_noisy, training=True)
        loss    = loss_fn(y, logits)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    acc = tf.reduce_mean(tf.cast(tf.argmax(logits,-1)==tf.argmax(y,-1), tf.float32))
    return loss, acc

for epoch in range(1, EPOCHS+1):
    tot_l, tot_a, n = 0., 0., 0
    for xb, yb in train_ds:
        l,a = train_step(xb, yb)
        bs  = xb.shape[0]
        tot_l += l.numpy()*bs; tot_a += a.numpy()*bs; n += bs
    print(f"Epoch {epoch:02d}: loss={tot_l/n:.4f}  noise-train acc={tot_a/n:.3%}")

model.save(CKPT_FILE)
print("✓ trained weights saved to", CKPT_FILE)

# ----------------------------------------------------------------------
# 4.  Certification utilities
# ----------------------------------------------------------------------
def clopper_low(k, n, alpha): return beta.ppf(alpha/2, k, n-k+1) if k>0 else 0.
def clopper_up (k, n, alpha): return beta.ppf(1-alpha/2, k+1, n-k) if k<n else 1.

@tf.function(jit_compile=True)
def predict_noisy_batch(x_batch, sigma):
    noise  = tf.random.normal(tf.shape(x_batch))*sigma
    logits = model(x_batch+noise, training=False)
    return tf.argmax(logits, axis=-1)          # int32

def smoothed_radius(x, sigma=SIGMA, N=N, N0=N0, alpha=ALPHA, batch=512):
    # Step-1: majority class
    counts = np.zeros(C, dtype=np.int32)
    reps   = tf.repeat(x[None,...], batch, axis=0)
    for done in range(0, N0, batch):
        m = min(batch, N0-done)
        preds = predict_noisy_batch(reps[:m], sigma)
        counts += np.bincount(preds.numpy(), minlength=C)
    A = counts.argmax()

    # Step-2: estimate p_A
    k_A = 0
    for done in range(0, N, batch):
        m = min(batch, N-done)
        preds = predict_noisy_batch(reps[:m], sigma)
        k_A += int(np.sum(preds.numpy()==A))
    pA_low = clopper_low(k_A, N, alpha)
    pB_up  = clopper_up (N-k_A, N, alpha)
    if pA_low<=pB_up:                   # certificate fails
        return 0.0
    return sigma * norm.ppf(pA_low)

def deterministic_radius(x, L=LIPSCHITZ):
    logits = model(x[None,...], training=False)
    top2   = tf.nn.top_k(logits, k=2).values
    margin = (top2[:,0]-top2[:,1])[0].numpy()   # scalar
    return margin/(2.0*L)

def hybrid_radius(x):
    r_det   = deterministic_radius(x)
    r_smooth= smoothed_radius(x)
    return max(r_det, r_smooth), r_det, r_smooth

# ----------------------------------------------------------------------
# 5.  Evaluate clean, deterministic, smoothing, and hybrid accuracies
# ----------------------------------------------------------------------
clean_ok, det_ok, smooth_ok, hybrid_ok = 0,0,0,0

for (x,y) in test_ds:
    true  = tf.argmax(y, -1).numpy()[0]
    pred  = tf.argmax(model(x, training=False), -1).numpy()[0]
    clean_ok += int(pred==true)

    r_h, r_d, r_s = hybrid_radius(tf.squeeze(x,0))
    det_ok    += int((r_d>0)    and (pred==true))
    smooth_ok += int((r_s>0)    and (pred==true))
    hybrid_ok += int((r_h>0)    and (pred==true))

N_TEST = len(x_te)
print("\n=== RESULTS (α = {:.0e}, σ = {:.2f}) ===".format(ALPHA,SIGMA))
print(f"Clean accuracy        : {clean_ok / N_TEST:.3%}")
print(f"Deterministic cert -r : {det_ok   / N_TEST:.3%}")
print(f"Smoothing    cert -r : {smooth_ok/ N_TEST:.3%}")
print(f"HYBRID       cert -r : {hybrid_ok/ N_TEST:.3%}")


In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from cleverhans.tf2.attacks.projected_gradient_descent import projected_gradient_descent

# 1) Make sure inputs are float32 in [0,1]
def to_float01(x):
    x = tf.convert_to_tensor(x)
    if x.dtype.is_integer:
        x = tf.cast(x, tf.float32) / 255.0
    else:
        x = tf.cast(x, tf.float32)
    return x

X_test_f = to_float01(X_test_s)
y_test_f = y_test_s
# 2) Labels as int (not one-hot) for metrics; keep one-hot separately if you want AUC-OVR
'''if 'y_test_one_hot' in globals():
    y_test_labels = np.argmax(y_test_one_hot, axis=1) if y_test_one_hot.ndim > 1 else np.asarray(y_test_one_hot).astype(int)
else:
    y_test_labels = np.asarray(y_test_s).astype(int)
'''
batch_size = 64
num_samples = len(X_test_f)
num_batches = (num_samples + batch_size - 1) // batch_size

epsilon_values = [0.016, 0.0313, 0.047, 0.0627]  # feel free to add more
results_per_epsilon = []

for epsilon in epsilon_values:
    adv_batches = []

    for i in range(num_batches):
        s = i * batch_size
        e = min((i + 1) * batch_size, num_samples)

        x_b = X_test_f[s:e]
        y_b = np.argmax(y_test_f[s:e], axis=1).astype("int32") #y_test_labels[s:e]

        # CleverHans expects tf.Tensors, float32, 0..1; set clip_min/max
        adv_b = projected_gradient_descent(
            model_fn=model,
            x=x_b,
            eps=tf.cast(epsilon, tf.float32),
            eps_iter=tf.cast(epsilon/4.0, tf.float32),
            nb_iter=10,                        # use >1 step for PGD (e.g., 40)
            norm=np.inf,
            loss_fn=None,
            clip_min=0.0,
            clip_max=1.0,
            y=tf.convert_to_tensor(y_b, dtype=tf.int32),
            targeted=False,
            rand_init=True,                    # typical PGD with random start
            rand_minmax=epsilon,
            sanity_checks=False,
        )
        adv_batches.append(adv_b.numpy())

    X_adv = np.concatenate(adv_batches, axis=0)

    # 3) Predict on adversarial set
    logits = model.predict(X_adv, batch_size=128, verbose=0)

    # 4) Handle binary vs multiclass automatically
    y_pred = tf.squeeze(logits)
    y_pred_binary = y_pred >= 0.5
    y_pred_binary = np.array(y_pred_binary, dtype='int32')

    # Calculate evaluation metrics for the current epsilon
    accuracy = accuracy_score(y_pred_binary, y_test_f) * 100
    precision = precision_score(y_pred_binary, y_test_f, average='macro') * 100
    recall = recall_score(y_pred_binary, y_test_f, average='macro') * 100
    f1 = f1_score(y_pred_binary, y_test_f, average='macro') * 100
    #auc = roc_auc_score(y_pred, y_test_one_hot) * 100

    # Store results for the current epsilon
    results_per_epsilon.append({
        'epsilon': epsilon,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        #'auc': auc
    })

# Print or use the results as needed
for result in results_per_epsilon:
    print(f"Epsilon: {result['epsilon']}")
    print(f"Accuracy: {result['accuracy']}")
    print(f"Precision: {result['precision']}")
    print(f"Recall: {result['recall']}")
    print(f"F1 Score: {result['f1']}")
    #print(f"AUC Score: {result['auc']}")
    print('-' * 50)

In [ ]:
y_pred = tf.squeeze(logits)
y_pred_binary = y_pred >= 0.5
y_pred_binary = np.array(y_pred_binary, dtype='int32')

# Calculate evaluation metrics for the current epsilon
accuracy = accuracy_score(y_pred_binary, y_test_f) * 100
precision = precision_score(y_pred_binary, y_test_f, average='macro') * 100
recall = recall_score(y_pred_binary, y_test_f, average='macro') * 100
f1 = f1_score(y_pred_binary, y_test_f, average='macro') * 100
#auc = roc_auc_score(y_pred, y_test_one_hot) * 100

# Store results for the current epsilon
results_per_epsilon.append({
    'epsilon': epsilon,
    'accuracy': accuracy,
    'precision': precision,
    'recall': recall,
    'f1': f1,
    #'auc': auc
})

# Print or use the results as needed
for result in results_per_epsilon:
    print(f"Epsilon: {result['epsilon']}")
    print(f"Accuracy: {result['accuracy']}")
    print(f"Precision: {result['precision']}")
    print(f"Recall: {result['recall']}")
    print(f"F1 Score: {result['f1']}")
    #print(f"AUC Score: {result['auc']}")
    print('-' * 50)


In [ ]:
y_pred = tf.squeeze(logits)
y_pred_binary = y_pred >= 0.5
y_pred_binary = np.array(y_pred_binary, dtype='int32')

# Calculate evaluation metrics for the current epsilon
accuracy = accuracy_score(y_pred_binary, y_test_f) * 100
precision = precision_score(y_pred_binary, y_test_f, average='macro') * 100
recall = recall_score(y_pred_binary, y_test_f, average='macro') * 100
f1 = f1_score(y_pred_binary, y_test_f, average='macro') * 100
#auc = roc_auc_score(y_pred, y_test_one_hot) * 100

# Store results for the current epsilon
results_per_epsilon.append({
    'epsilon': epsilon,
    'accuracy': accuracy,
    'precision': precision,
    'recall': recall,
    'f1': f1,
    #'auc': auc
})

# Print or use the results as needed
for result in results_per_epsilon:
    print(f"Epsilon: {result['epsilon']}")
    print(f"Accuracy: {result['accuracy']}")
    print(f"Precision: {result['precision']}")
    print(f"Recall: {result['recall']}")
    print(f"F1 Score: {result['f1']}")
    #print(f"AUC Score: {result['auc']}")
    print('-' * 50)
